This notebook demonstrates how we collected and normalized SpaceX launch data via REST calls.




In [3]:
#  Imports & Config
import requests
import pandas as pd

BASE_URL = "https://api.spacexdata.com/v4"



In [4]:
#  Fetch all launches
resp = requests.get(f"{BASE_URL}/launches")
launches = resp.json()

# Normalize into a DataFrame
df_launches = pd.json_normalize(launches)
df_launches.head()


,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN


Outcome

In [5]:
#  For the first 5 launches, pull detailed sub-resources
details = []
for idx, row in df_launches.head(5).iterrows():
    lid = row['id']
    data = requests.get(f"{BASE_URL}/launches/{lid}").json()
    # pull rocket name
    rock = requests.get(f"{BASE_URL}/rockets/{data['rocket']}").json()['name']
    data['rocket_name'] = rock
    details.append(data)

df_details = pd.json_normalize(details)
df_details[['name','date_utc','success','rocket_name']].head()


,name,date_utc,success,rocket_name
0,FalconSat,2006-03-24T22:30:00.000Z,False,Falcon 1
1,DemoSat,2007-03-21T01:10:00.000Z,False,Falcon 1
2,Trailblazer,2008-08-03T03:34:00.000Z,False,Falcon 1
3,RatSat,2008-09-28T23:15:00.000Z,True,Falcon 1
4,RazakSat,2009-07-13T03:35:00.000Z,True,Falcon 1


In [6]:
#  Save to CSV for downstream work
df_details.to_csv("spacex_launches_sample.csv", index=False)
print("Saved sample to spacex_launches_sample.csv")


Saved sample to spacex_launches_sample.csv
